# Further analysis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

# Assuming you've already run the previous functions to create:
# - stable_timeseries = create_stable_point_timeseries(timeseries)
# - features_df = extract_ml_features(stable_timeseries)

# 1. Check feature distribution and handle missing values
print(f"Feature dataset shape: {features_df.shape}")
print(f"Missing values: {features_df.isnull().sum().sum()}")

# Fill missing values with median
features_df = features_df.fillna(features_df.median())

# 2. Prepare features and target
X = features_df.drop(['point_id', 'point_lon', 'point_lat', 'is_damaged'], axis=1)
y = features_df['is_damaged']

# 3. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"Class distribution in training: {np.bincount(y_train)}")
print(f"Class distribution in testing: {np.bincount(y_test)}")

# 4. Create a pipeline with preprocessing and model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 5. Define hyperparameters for grid search
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# 6. Perform cross-validation and grid search
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    pipeline, param_grid, cv=cv, scoring='f1', verbose=1, n_jobs=-1
)

# Fit the model
grid_search.fit(X_train, y_train)

# 7. Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# 8. Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# 9. Print classification metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 10. Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Undamaged', 'Damaged'],
            yticklabels=['Undamaged', 'Damaged'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# 11. Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# 12. Plot Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

# 13. Feature importance analysis
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': best_model.named_steps['classifier'].feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances.head(20))
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

# 14. Analyze errors
errors = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'probability': y_pred_proba,
    'error': y_test != y_pred
})

# Add original point coordinates
errors = errors.reset_index().merge(
    features_df[['point_id', 'point_lon', 'point_lat']], 
    left_on='index', right_index=True, how='left'
)

# False positives (predicted damaged but actually undamaged)
false_positives = errors[(errors['error']) & (errors['predicted'] == 1)]
print(f"\nFalse positives: {len(false_positives)} points")

# False negatives (predicted undamaged but actually damaged)
false_negatives = errors[(errors['error']) & (errors['predicted'] == 0)]
print(f"False negatives: {len(false_negatives)} points")

# Save error analysis for potential visual inspection
false_positives.to_csv('false_positives.csv', index=False)
false_negatives.to_csv('false_negatives.csv', index=False)

# 15. Save the trained model
import joblib
joblib.dump(best_model, 'damage_detection_model.pkl')

print("\nModel training and evaluation complete!")